# Test: open all ageing; GITT; contact resistance; hystries etc. 

In [1]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2

In [2]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
import scipy.optimize
import random;import time, signal
fs=10;
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2 import * 
# pb.set_logging_level("INFO")

In [3]:
str_path_3 = os.path.abspath(os.path.join(pb.__path__[0],'..\..'))
str_path_4 = os.path.abspath(os.path.join(str_path_3,"Lib\site-packages\klu"))
print(str_path_4)
import klu

d:\gitpybamm\lrh_409\lrh_409\envpbgem\Lib\site-packages\klu


In [4]:
import importlib
idaklu_spec = importlib.util.find_spec("pybamm.solvers.casadi_solver")
print(idaklu_spec)

ModuleSpec(name='pybamm.solvers.casadi_solver', loader=<_frozen_importlib_external.SourceFileLoader object at 0x00000220B6105550>, origin='d:\\gitpybamm\\lrh_409\\lrh_409\\envpbgem\\pybamm-gem\\pybamm\\solvers\\casadi_solver.py')


In [ ]:
import importlib
idaklu_spec = importlib.util.find_spec("pybamm.solvers.casadi")
print(idaklu_spec)
sim2 = pb.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    #solver=pb.CasadiSolver(return_solution_if_failed_early=True),
    solver=pb.IDAKLUSolver(),
    var_pts=var_pts,
)
sol_0 = sim2.solve()

In [ ]:
sys.path.remove("d:\softwarelrh\python",)
sys.path.remove("d:\softwarelrh\python\lib",)
sys.path.remove("d:\softwarelrh\python\DLLs",)
sys.path.remove("d:\softwarelrh\python\python39.zip",)

In [ ]:
#print(sys.path[0])
for i in range(len(sys.path)):
    print(sys.path[i])

In [ ]:
model = pb.lithium_ion.DFN(
    options={
    "thermal": "lumped",
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "particle mechanics": ("swelling and cracking", "swelling only"),
    "loss of active material": "stress-driven",
    "lithium plating": "partially reversible",
    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
} 
)
param = pb.ParameterValues("OKane2023")
# param.update({"Negative electrode porosity": 0.1})
# Adjust SEI parameters
param.update({"Inner SEI lithium interstitial diffusivity [m2.s-1]": 3e-19})
# Override SEI thickness dependence of lithium plating
param.update({"Dead lithium decay rate [s-1]": 1e-8})
param.update({"Lithium plating kinetic rate constant [m.s-1]": 1e-12})
# Adjust mechanical degradation parameters
param.update({"Negative electrode cracking rate": 1e-32})
param.update({"Negative electrode LAM constant proportional term [s-1]": 1e-19})
param.update({"Positive electrode LAM constant proportional term [s-1]": 1e-19})

c_e = model.variables["Electrolyte concentration [mol.m-3]"]
T = model.variables["Cell temperature [K]"]
D_e = model.param.D_e
kappa_e = model.param.kappa_e
model.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e, T)
model.variables["Electrolyte conductivity [S.m-1]"] = kappa_e(c_e, T)
# use electrolyte
V_max = 4.2;        V_min = 2.5; 
exp_RPT_0p1C_text = [ (
    f"Discharge at 1 C until {V_min} V (1 minute period)", #  (1 minute period)  
    "Rest for 1 hours (5 minute period)",  
    f"Charge at 1 C until {V_max} V (1 minute period)",
    "Rest for 1 hours (5 minute period)",
    ) ]
exp_RPT_refill_text = [ (
    f"Charge at 0.3C until {V_max} V",  
    f"Hold at {V_max}V until C/100",
    "Rest for 1 hours (5 minute period)",  
    ) ]
exp_RPT_GITT_text = [ (
    "Rest for 60 seconds (0.1 second period)",  
    f"Discharge at C/2 for 4.8 minutes or until {V_min}V (0.1 second period)",
    "Rest for 1 hour (5 minute period)",  
    ) ]
# how to get 0.1s Resistance: voltage: mean of rest step [-10:-1] data points; don't include the last data points of rest step!;
# should also get OCV and SOC 
Experiment_Breakin= pb.Experiment( exp_RPT_0p1C_text * 1  ) # +  exp_RPT_refill_text*1 +   exp_RPT_GITT_text*24
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 60,  # negative particle
    "r_p": 30,  # positive particle
}


In [ ]:
# CasadiSolver
param.update({"Electrolyte conductivity [S.m-1]": electrolyte_conductivity_EC_EMC_3_7_Landesfeind2019_Constant})
param.update({"Electrolyte diffusivity [m2.s-1]": electrolyte_diffusivity_EC_EMC_3_7_Landesfeind2019_Constant})
param.update({"Thermodynamic factor": electrolyte_TDF_EC_EMC_3_7_Landesfeind2019_Constant})
param.update({"Cation transference number": electrolyte_transference_number_EC_EMC_3_7_Landesfeind2019_Constant})
sim2 = pb.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pb.CasadiSolver(return_solution_if_failed_early=True),
    var_pts=var_pts,
)
sol_0 = sim2.solve()

# try IDAKLUSolver
sim3 = pb.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pb.IDAKLUSolver(),
    var_pts=var_pts,
)
sol_1 = sim3.solve()

In [ ]:
model.variables.search("LAM")

In [ ]:
#model.variables.search("SEI")
label = ["Casadi","IDAKLU"] 
output_variables3 = [
    "Electrolyte potential [V]",
    "Loss of capacity to SEI on cracks [A.h]",
    "Current [A]", 
    "Terminal voltage [V]", 
    "Loss of capacity to SEI [A.h]",
    "LAM_ne [%]",+
    "LAM_pe [%]",
]
quick_plot = pb.QuickPlot(
    [sol_0,sol_1], output_variables3,label,variable_limits='fixed',
    time_unit='hours',n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()

In [ ]:
# write a function to get the discharge capacity and resistance
def Get_0p1s_R0(sol_RPT,Index,cap_full):
    Res_0p1s = []; SOC = [100,];
    for i,index in enumerate(Index):
        cycle = sol_RPT.cycles[index]
        Res_0p1s.append(   (
            np.mean(cycle.steps[0]["Terminal voltage [V]"].entries[-10:-1])
            - cycle.steps[1]["Terminal voltage [V]"].entries[0]
        ) / cycle.steps[1]["Current [A]"].entries[0] * 1000)
        if i > 0:
            Dis_Cap = abs(
                cycle.steps[1]["Discharge capacity [A.h]"].entries[0] 
                - cycle.steps[1]["Discharge capacity [A.h]"].entries[-1] )
            SOC.append(SOC[-1]-Dis_Cap/cap_full*100)
    return np.mean(Res_0p1s),Res_0p1s,SOC
cap_full = 5;
Index = np.arange(2,26,1) # index = 2:25
Mean_Res_0p1s,Res_0p1s,SOC = Get_0p1s_R0(sol_0,Index,cap_full)

fs=14;
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

fig, ax = plt.subplots( figsize=(5,4),tight_layout=True)
ax.plot(SOC,Res_0p1s,"-o")
ax.set_xlabel("SOC %")
ax.set_ylabel(r'Mean 0.1s R (m$\Omega$)')
print(SOC[-1],Mean_Res_0p1s)

In [ ]:
label = [
    "Step-0",
    "Step-1",
    "Step-2",] 
var = [
    "Terminal voltage [V]",
    "Current [A]",
    "Electrolyte concentration [mol.m-3]",
    "Electrolyte diffusivity [m2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "X-averaged SEI interfacial current density [A.m-2]",
]
quick_plot = pb.QuickPlot(
    [
    sol_0.cycles[0],
    sol_0.cycles[1],
    sol_0.cycles[2],
    ], var,label,variable_limits='tight') #     
quick_plot.dynamic_plot();

In [ ]:
len(sol_0.cycles)

In [ ]:

var = [
    "Terminal voltage [V]",
    "Current [A]",
    "Electrolyte concentration [mol.m-3]",
    "Electrolyte diffusivity [m2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "X-averaged SEI interfacial current density [A.m-2]",
]
quick_plot = pb.QuickPlot(
    [cycle for cycle in sol_0.cycles[2:-1]],
    var,variable_limits='tight') #     
quick_plot.dynamic_plot();